<a href="https://colab.research.google.com/github/leticiarccorrea/sales-operations-demand-forecasting/blob/main/case_salesmodel_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import dataset

from google.colab import drive
import pandas as pd


# Access to Google Drive
drive.mount('/content/drive')
datapah = '/content/drive/MyDrive/caseboti/dataset.csv'
datapah_dic = '/content/drive/MyDrive/caseboti/dicionariodedados.csv'


# Load file in pandas and spark
base = pd.read_csv(datapah, sep=';', on_bad_lines='warn')
base_dictionary = pd.read_csv(datapah_dic)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# import de bibliotecas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional, Iterable

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error


from typing import Dict

import warnings
warnings.filterwarnings('ignore')
from typing import List

import numpy as np
import pandas as pd

from dataclasses import dataclass
from typing import List, Dict, Tuple

import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

import lightgbm as lgb

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

from typing import Dict, List, Tuple
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error


**1. Preparação do dataset**

In [3]:
# Imports
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional


# ============================
# Config (model input)
# ============================
@dataclass
class ModelInputConfig:
    # Comentário: colunas obrigatórias do dataset bruto
    required_cols: Tuple[str, ...] = (
        "COD_CICLO",
        "COD_MATERIAL",
        "COD_CANAL",
        "COD_REGIAO",
        "QT_VENDA_BRUTO",
        "QT_DEVOLUCAO",
        "PCT_DESCONTO",
        "VL_PRECO",
    )

    # Comentário: colunas numéricas que podem vir como string PT-BR/EN
    numeric_like_cols: Tuple[str, ...] = (
        "QT_VENDA_BRUTO",
        "QT_DEVOLUCAO",
        "VL_RECEITA_BRUTA",
        "VL_RECEITA_LIQUIDA",
        "PCT_DESCONTO",
        "VL_PRECO",
    )

    # Comentário: regras de desconto
    discount_clip: Tuple[float, float] = (0.0, 1.0)

    # Comentário: regras de target
    default_devolution: float = 0.0
    min_target_value: float = 0.0  # Comentário: para input do modelo, removemos vendas líquidas negativas

    # Comentário: filtro de qualidade temporal
    drop_invalid_ciclo: bool = True

    # Comentário: imputação de preço SEM leakage (expanding median por SKU)
    price_fallback_to_global_median: bool = True

    # Comentário: histórico mínimo por série para gerar lags (ex.: lag_6 precisa >= 7 pontos)
    series_keys: Tuple[str, ...] = ("cod_material", "cod_canal", "cod_regiao")
    min_history_per_series: int = 8

    # Comentário: remove séries com pouca venda (reduz ruído/zeros)
    min_positive_sales_points: int = 3


# ============================
# Helpers
# ============================
def parse_ptbr_number(series: pd.Series) -> pd.Series:
    """
    Converte números em formato PT-BR (1.234,56) ou EN (1234.56) para float.
    Comentário: só remove '.' como milhar quando existir ',' (indicando PT-BR).
    """
    if pd.api.types.is_numeric_dtype(series):
        return series

    s = (
        series.astype(str)
        .str.strip()
        .replace(
            {
                "": np.nan,
                "nan": np.nan,
                "NaN": np.nan,
                "NULL": np.nan,
                "None": np.nan,
            }
        )
    )

    mask_ptbr = s.str.contains(",", na=False)
    s.loc[mask_ptbr] = (
        s.loc[mask_ptbr]
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )

    return pd.to_numeric(s, errors="coerce")


def rename_to_snake_case(df: pd.DataFrame) -> pd.DataFrame:
    """
    Comentário: renomeia colunas principais para snake_case.
    """
    rename_map = {
        "COD_CICLO": "cod_ciclo",
        "COD_MATERIAL": "cod_material",
        "COD_CANAL": "cod_canal",
        "COD_REGIAO": "cod_regiao",
        "DES_CATEGORIA_MATERIAL": "des_categoria_material",
        "DES_MARCA_MATERIAL": "des_marca_material",
        "QT_VENDA_BRUTO": "qt_venda_bruto",
        "QT_DEVOLUCAO": "qt_devolucao",
        "VL_RECEITA_BRUTA": "vl_receita_bruta",
        "VL_RECEITA_LIQUIDA": "vl_receita_liquida",
        "PCT_DESCONTO": "pct_desconto",
        "VL_PRECO": "vl_preco",
        "FLG_DATA": "flg_data",
        "FLG_CAMPANHA_MKT_A": "flg_campanha_mkt_a",
        "FLG_CAMPANHA_MKT_B": "flg_campanha_mkt_b",
        "FLG_CAMPANHA_MKT_C": "flg_campanha_mkt_c",
        "FLG_CAMPANHA_MKT_D": "flg_campanha_mkt_d",
        "FLG_CAMPANHA_MKT_E": "flg_campanha_mkt_e",
    }
    return df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})


def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Comentário: cria ano e ciclo a partir de cod_ciclo no formato YYYYCC.
    """
    out = df.copy()
    cod_ciclo_num = pd.to_numeric(out["cod_ciclo"], errors="coerce")
    out["ano"] = (cod_ciclo_num // 100).astype("Int64")
    out["ciclo"] = (cod_ciclo_num % 100).astype("Int64")
    return out


def build_campaign_flag(df: pd.DataFrame) -> pd.DataFrame:
    """
    Comentário: cria flg_campanha_resumo a partir de colunas flg_campanha* (se existirem).
    """
    out = df.copy()
    campaign_cols = [c for c in out.columns if c.startswith("flg_campanha")]

    if len(campaign_cols) > 0:
        out["flg_campanha_resumo"] = (
            out[campaign_cols]
            .fillna(0)
            .max(axis=1)
            .astype(int)
        )
    else:
        out["flg_campanha_resumo"] = 0

    return out


def normalize_discount(df: pd.DataFrame, cfg: ModelInputConfig) -> pd.DataFrame:
    """
    Comentário: garante desconto em 0..1, assumindo 0 quando ausente e convertendo % quando necessário.
    """
    out = df.copy()
    if "pct_desconto" not in out.columns:
        out["pct_desconto"] = 0.0
        return out

    out["pct_desconto"] = out["pct_desconto"].fillna(0.0)

    if out["pct_desconto"].max(skipna=True) > 1.0:
        out["pct_desconto"] = out["pct_desconto"] / 100.0

    out["pct_desconto"] = out["pct_desconto"].clip(cfg.discount_clip[0], cfg.discount_clip[1])
    return out


def compute_targets(df: pd.DataFrame, cfg: ModelInputConfig) -> pd.DataFrame:
    """
    Comentário: cria demanda líquida e pct_devolucao.
    """
    out = df.copy()

    out["qt_devolucao"] = out.get("qt_devolucao", cfg.default_devolution)
    out["qt_devolucao"] = pd.to_numeric(out["qt_devolucao"], errors="coerce").fillna(cfg.default_devolution)

    out["qt_venda_bruto"] = out.get("qt_venda_bruto", 0.0)
    out["qt_venda_bruto"] = pd.to_numeric(out["qt_venda_bruto"], errors="coerce").fillna(0.0)

    out["qt_venda_liquida"] = out["qt_venda_bruto"] - out["qt_devolucao"]

    out["pct_devolucao"] = np.where(
        out["qt_venda_bruto"] > 0,
        out["qt_devolucao"] / out["qt_venda_bruto"],
        np.nan,
    )

    return out


def impute_price_expanding_median_by_sku(
    df: pd.DataFrame,
    sku_col: str = "cod_material",
    time_col: str = "cod_ciclo",
    price_col: str = "vl_preco",
    fallback_to_global: bool = True,
) -> pd.DataFrame:
    """
    Comentário: imputa vl_preco faltante usando mediana expanding por SKU (sem usar futuro).
    """
    out = df.copy()
    if sku_col not in out.columns or time_col not in out.columns:
        return out

    if price_col not in out.columns:
        out[price_col] = np.nan

    out = out.sort_values([sku_col, time_col]).reset_index(drop=True)

    expanding_median = (
        out.groupby(sku_col, observed=True)[price_col]
        .expanding()
        .median()
        .reset_index(level=0, drop=True)
    )

    out[price_col] = pd.to_numeric(out[price_col], errors="coerce")
    out[price_col] = out[price_col].fillna(expanding_median)

    if fallback_to_global:
        global_median = out[price_col].median(skipna=True)
        out[price_col] = out[price_col].fillna(global_median)

    return out


def filter_series_for_modeling(
    df: pd.DataFrame,
    series_keys: Tuple[str, ...],
    time_col: str,
    target_col: str,
    min_history: int,
    min_positive_points: int,
) -> pd.DataFrame:
    """
    Comentário: remove séries com histórico insuficiente e com poucas vendas > 0.
    """
    out = df.copy()

    # Comentário: histórico por série
    history = out.groupby(list(series_keys), observed=True)[time_col].nunique()
    keep_history = history[history >= min_history].index

    out = out.set_index(list(series_keys))
    out = out.loc[out.index.isin(keep_history)].reset_index()

    # Comentário: remove séries muito esparsas (poucos pontos de venda positiva)
    pos_points = out.groupby(list(series_keys), observed=True)[target_col].apply(lambda s: int((s > 0).sum()))
    keep_pos = pos_points[pos_points >= min_positive_points].index

    out = out.set_index(list(series_keys))
    out = out.loc[out.index.isin(keep_pos)].reset_index()

    return out


# ============================
# Main: build model input
# ============================
def build_model_input_dataset(
    base: pd.DataFrame,
    cfg: Optional[ModelInputConfig] = None,
) -> Tuple[pd.DataFrame, Dict[str, object]]:
    """
    Comentário: gera dataset final pronto para modelagem (sem NaNs críticos).
    Retorna:
      - model_df: dataset pronto para criar lags/rolling e treinar
      - report: métricas de limpeza e filtros aplicados
    """
    if cfg is None:
        cfg = ModelInputConfig()

    # Comentário: valida schema mínimo
    missing = [c for c in cfg.required_cols if c not in base.columns]
    if missing:
        raise ValueError(f"Missing required columns in base: {missing}")

    df = base.copy()

    # Comentário: conversão numérica no schema original (UPPER)
    for col in cfg.numeric_like_cols:
        if col in df.columns:
            df[col] = parse_ptbr_number(df[col])

    # Comentário: rename para snake_case
    df = rename_to_snake_case(df)

    # Comentário: desconto e targets
    df = normalize_discount(df, cfg)
    df = compute_targets(df, cfg)

    # Comentário: tempo e campanha
    df = add_time_features(df)
    df = build_campaign_flag(df)

    # Comentário: imputação de preço sem leakage
    df = impute_price_expanding_median_by_sku(
        df=df,
        sku_col="cod_material",
        time_col="cod_ciclo",
        price_col="vl_preco",
        fallback_to_global=cfg.price_fallback_to_global_median,
    )

    # Comentário: report inicial
    report: Dict[str, object] = {
        "rows_raw": int(len(base)),
        "rows_after_numeric_and_targets": int(len(df)),
        "missing_rate_vl_preco": float(df["vl_preco"].isna().mean()) if "vl_preco" in df.columns else None,
        "missing_rate_cod_ciclo": float(df["cod_ciclo"].isna().mean()) if "cod_ciclo" in df.columns else None,
    }

    # Comentário: remove ciclos inválidos (input do modelo precisa de tempo válido)
    invalid_ciclo_mask = df["ano"].isna() | df["ciclo"].isna()
    report["rows_invalid_ciclo"] = int(invalid_ciclo_mask.sum())

    if cfg.drop_invalid_ciclo:
        df = df.loc[~invalid_ciclo_mask].copy()

    # Comentário: remove targets negativos (para demanda, normalmente não faz sentido no modelo)
    neg_target_mask = df["qt_venda_liquida"] < cfg.min_target_value
    report["rows_negative_target"] = int(neg_target_mask.sum())
    df = df.loc[~neg_target_mask].copy()

    # Comentário: remove NaNs críticos para modelagem
    critical_cols = [
        "cod_ciclo", "cod_material", "cod_canal", "cod_regiao",
        "qt_venda_liquida", "vl_preco", "pct_desconto",
        "flg_campanha_resumo",
    ]
    critical_cols = [c for c in critical_cols if c in df.columns]

    before_dropna = len(df)
    df = df.dropna(subset=critical_cols).copy()
    report["rows_dropped_by_critical_nan"] = int(before_dropna - len(df))

    # Comentário: garante tipos finais
    df["cod_ciclo"] = pd.to_numeric(df["cod_ciclo"], errors="coerce").astype("Int64")
    df["vl_preco"] = pd.to_numeric(df["vl_preco"], errors="coerce")
    df["pct_desconto"] = pd.to_numeric(df["pct_desconto"], errors="coerce")
    df["qt_venda_liquida"] = pd.to_numeric(df["qt_venda_liquida"], errors="coerce")

    # Comentário: ordena (essencial para lags/rolling)
    df = df.sort_values(list(cfg.series_keys) + ["cod_ciclo"]).reset_index(drop=True)

    # Comentário: filtra séries com histórico mínimo
    before_series_filter = len(df)
    df = filter_series_for_modeling(
        df=df,
        series_keys=cfg.series_keys,
        time_col="cod_ciclo",
        target_col="qt_venda_liquida",
        min_history=cfg.min_history_per_series,
        min_positive_points=cfg.min_positive_sales_points,
    )
    report["rows_dropped_by_series_filter"] = int(before_series_filter - len(df))

    report["rows_final"] = int(len(df))
    report["n_series_final"] = int(df[list(cfg.series_keys)].drop_duplicates().shape[0])
    report["cycles_final"] = int(df["cod_ciclo"].nunique())

    return df, report


# ============================
# Run (final)
# ============================
cfg = ModelInputConfig(
    drop_invalid_ciclo=True,              # Comentário: modelo precisa de tempo válido
    min_target_value=0.0,                 # Comentário: remove demanda líquida negativa
    min_history_per_series=8,             # Comentário: suporta lags até 6 com folga
    min_positive_sales_points=3,          # Comentário: reduz séries quase sempre zero
    price_fallback_to_global_median=True, # Comentário: último recurso
)

model_df, model_report = build_model_input_dataset(base, cfg=cfg)

print(model_report)
display(model_df.head())


{'rows_raw': 173923, 'rows_after_numeric_and_targets': 173923, 'missing_rate_vl_preco': 0.0, 'missing_rate_cod_ciclo': 0.0, 'rows_invalid_ciclo': 0, 'rows_negative_target': 154, 'rows_dropped_by_critical_nan': 0, 'rows_dropped_by_series_filter': 5588, 'rows_final': 168181, 'n_series_final': 5002, 'cycles_final': 53}


,cod_material,cod_canal,cod_regiao,cod_ciclo,flg_data,des_categoria_material,des_marca_material,qt_venda_bruto,qt_devolucao,vl_receita_bruta,...,flg_campanha_mkt_c,flg_campanha_mkt_d,flg_campanha_mkt_e,pct_desconto,vl_preco,qt_venda_liquida,pct_devolucao,ano,ciclo,flg_campanha_resumo
0,6480,anon_S7,anon_S1,201801,0,anon_S12,anon_S17,276.0,0.0,4429.8,...,0,0,0,0.0,833.4,276.0,0.0,2018,1,0
1,6480,anon_S7,anon_S1,201802,0,anon_S12,anon_S17,288.0,0.0,5628.6,...,0,0,0,0.0,833.4,288.0,0.0,2018,2,0
2,6480,anon_S7,anon_S1,201803,0,anon_S12,anon_S17,258.0,0.0,2631.6,...,0,0,0,0.0,833.4,258.0,0.0,2018,3,0
3,6480,anon_S7,anon_S1,201804,0,anon_S12,anon_S17,282.0,0.0,5029.2,...,0,0,0,0.0,833.4,282.0,0.0,2018,4,0
4,6480,anon_S7,anon_S1,201805,0,anon_S12,anon_S17,270.0,0.0,3830.4,...,0,0,0,0.0,833.4,270.0,0.0,2018,5,0


In [4]:
len(model_df)

168181

In [5]:
base_work = model_df

**2. Definições de série e granularidade**

série = (`cod_material`, `cod_canal`, `cod_regiao`)

Motivo: para estimar o plano logístico é necessário o detalhamento por canal/região e, para itens críticos, por SKU.

Esse grão permite:
- capturar heterogeneidade estrutural (volume, elasticidade a preço/desconto, resposta a campanha),
- fazer agregações coerentes para níveis superiores (categoria/marca/região).



In [6]:
series_keys = ["cod_material", "cod_canal", "cod_regiao"]
time_key = "cod_ciclo"
target_col = "qt_venda_liquida"

# Ordenação temporal
base_work = base_work.sort_values(series_keys + [time_key]).reset_index(drop=True)

# Algumas checagens rápidas
print("linhas:", len(base_work))
print("n_séries:", base_work[series_keys].drop_duplicates().shape[0])
print("ciclos:", base_work[time_key].nunique(), "| min:", base_work[time_key].min(), "| max:", base_work[time_key].max())

linhas: 168181
n_séries: 5002
ciclos: 53 | min: 201801 | max: 202101


**3. Engenharia de features para painel temporal (global model)**


Estratégia:
- **Lags** da demanda (ex.: 1, 2, 3 ciclos)
- **Rolling stats** (média/mediana/std móveis)
- **Sazonalidade** via decomposição simples do código do ciclo (ano, índice dentro do ano)
- Features de preço/desconto e flags (campanha, data comemorativa)

Isso evita o problema de treinar um modelo por SKU (muito caro e instável) e aproveita sinal cruzado entre séries.


In [7]:
def split_ciclo_components(cod_ciclo: pd.Series) -> pd.DataFrame:
    """
    Extrai componentes do COD_CICLO (formato AAAACC).
    obs: não assumi 'ciclos por ano'; usei apenas:
      - ciclo_year: AAAA
      - ciclo_index: CC (índice do ciclo dentro do ano, sem trigonometria) - sem informação no texto base
    """
    s = pd.to_numeric(cod_ciclo, errors="coerce").astype("Int64").astype(str).str.zfill(6)
    ciclo_year = s.str.slice(0, 4).astype("Int64")
    ciclo_index = s.str.slice(4, 6).astype("Int64")
    return pd.DataFrame({"ciclo_year": ciclo_year, "ciclo_index": ciclo_index})


def add_lag_rolling_features(
    df: pd.DataFrame,
    group_keys: List[str],
    time_key: str,
    target_col: str,
    lags: List[int],
    rolling_windows: List[int],
) -> pd.DataFrame:
    """
    Cria lags e estatísticas móveis dentro de cada série.
    """
    # ordenação é obrigatória para não embaralhar lags/rolling
    out = df.sort_values(group_keys + [time_key]).copy()

    # componentes simples do ciclo
    cyc = split_ciclo_components(out[time_key])
    out = pd.concat([out.reset_index(drop=True), cyc.reset_index(drop=True)], axis=1)

    g = out.groupby(group_keys, observed=True)

    # lags da demanda
    for lag in lags:
        out[f"{target_col}_lag_{lag}"] = g[target_col].shift(lag)

    # rolling stats SEM leakage (usa shift(1))
    shifted_y = g[target_col].shift(1)
    group_id = g.ngroup()

    for w in rolling_windows:
        min_p = max(2, w // 2)

        out[f"{target_col}_roll_mean_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).mean()
        )
        out[f"{target_col}_roll_std_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).std()
        )
        out[f"{target_col}_roll_median_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).median()
        )
        out[f"{target_col}_roll_min_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).min()
        )
        out[f"{target_col}_roll_max_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).max()
        )

        # volatilidade relativa (CV = std/mean)
        out[f"{target_col}_roll_cv_{w}"] = (
            out[f"{target_col}_roll_std_{w}"] / out[f"{target_col}_roll_mean_{w}"].replace(0, np.nan)
        )

        # share de zeros na janela (demanda intermitente)
        out[f"{target_col}_roll_zero_share_{w}"] = shifted_y.groupby(group_id).transform(
            lambda s: s.rolling(window=w, min_periods=min_p).apply(lambda x: np.mean(np.array(x) == 0), raw=False)
        )

    #  tendência curta (sem leakage, usando lags)
    if f"{target_col}_lag_1" in out.columns and f"{target_col}_lag_2" in out.columns:
        out[f"{target_col}_diff_1"] = out[f"{target_col}_lag_1"] - out[f"{target_col}_lag_2"]
        out[f"{target_col}_pct_change_1"] = (
            out[f"{target_col}_diff_1"] / out[f"{target_col}_lag_2"].replace(0, np.nan)
        )

    # interações e dinâmica de preço/desconto (sem leakage via lag_1)
    if "vl_preco" in out.columns and "pct_desconto" in out.columns:
        out["vl_preco_efetivo"] = out["vl_preco"] * (1.0 - out["pct_desconto"].clip(0, 1))
        out["vl_preco_x_desconto"] = out["vl_preco"] * out["pct_desconto"].fillna(0)

        out["vl_preco_lag_1"] = g["vl_preco"].shift(1)
        out["pct_desconto_lag_1"] = g["pct_desconto"].shift(1)

        out["vl_preco_pct_change_1"] = (out["vl_preco"] / out["vl_preco_lag_1"]) - 1.0
        out["pct_desconto_change_1"] = out["pct_desconto"] - out["pct_desconto_lag_1"]

    return out


# Aplicação
lags = [1, 2, 3]
rolling_windows = [2, 5]

feat_df = add_lag_rolling_features(
    df=base_work,
    group_keys=series_keys,
    time_key=time_key,
    target_col=target_col,
    lags=lags,
    rolling_windows=rolling_windows,
)

# features candidatas (sem sin/cos)
candidate_features = [
    "ciclo_year", "ciclo_index",
    "pct_desconto", "vl_preco", "vl_preco_efetivo", "vl_preco_x_desconto",
    "vl_preco_pct_change_1", "pct_desconto_change_1",
    "flg_campanha_resumo", "flg_data",
    f"{target_col}_diff_1", f"{target_col}_pct_change_1",
]

candidate_features += [c for c in feat_df.columns if c.startswith(f"{target_col}_lag_")]
candidate_features += [c for c in feat_df.columns if c.startswith(f"{target_col}_roll_")]

feature_cols = [c for c in candidate_features if c in feat_df.columns]

feat_df[feature_cols + [target_col]].head()


,ciclo_year,ciclo_index,pct_desconto,vl_preco,vl_preco_efetivo,vl_preco_x_desconto,vl_preco_pct_change_1,pct_desconto_change_1,flg_campanha_resumo,flg_data,...,qt_venda_liquida_roll_cv_2,qt_venda_liquida_roll_zero_share_2,qt_venda_liquida_roll_mean_5,qt_venda_liquida_roll_std_5,qt_venda_liquida_roll_median_5,qt_venda_liquida_roll_min_5,qt_venda_liquida_roll_max_5,qt_venda_liquida_roll_cv_5,qt_venda_liquida_roll_zero_share_5,qt_venda_liquida
0,2018,1,0.0,833.4,833.4,0.0,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,276.0
1,2018,2,0.0,833.4,833.4,0.0,0.0,0.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288.0
2,2018,3,0.0,833.4,833.4,0.0,0.0,0.0,0,0,...,0.030090,0.0,282.0,8.485281,282.0,276.0,288.0,0.030090,0.0,258.0
3,2018,4,0.0,833.4,833.4,0.0,0.0,0.0,0,0,...,0.077704,0.0,274.0,15.099669,276.0,258.0,288.0,0.055108,0.0,282.0
4,2018,5,0.0,833.4,833.4,0.0,0.0,0.0,0,0,...,0.062854,0.0,276.0,12.961481,279.0,258.0,288.0,0.046962,0.0,270.0


**4. Estrutura de validação: walk-forward (expanding window) com horizonte > 1**

Para simular deploy real:
- Em cada *split*, treinamos com ciclos até `t` e prevemos `t+1..t+h`.
- Não há embaralhamento e não é usado informação futura.

Estratégia para `horizon=1..3`: **direct multi-horizon**
- Treina-se um modelo por horizonte (h=1,2,3) com o alvo deslocado `shift(-h)`.
- Isso tende a ser mais estável do que recursivo quando há volatilidade.

In [8]:
@dataclass
class WalkForwardConfig:
    min_train_cycles: int = 20   # mínimo de ciclos no treino
    step_size: int = 1           # avança 1 ciclo por split
    test_cycles: int = 3         # horizonte máximo avaliado
    max_splits: int = 6          # limita splits para performance (ajuste conforme necessidade)


def iter_walk_forward_splits(
    unique_cycles: np.ndarray,
    cfg: WalkForwardConfig,
) -> Iterable[Tuple[np.ndarray, np.ndarray]]:
    """Gera pares (train_cycles, test_cycles) para walk-forward expanding window."""
    cycles = np.array(sorted(unique_cycles))
    start = cfg.min_train_cycles
    end = len(cycles) - cfg.test_cycles

    split_count = 0
    for i in range(start, end + 1, cfg.step_size):
        train_cycles = cycles[:i]
        test_cycles = cycles[i : i + cfg.test_cycles]
        yield train_cycles, test_cycles
        split_count += 1
        if split_count >= cfg.max_splits:
            break


def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    denom = np.sum(np.abs(y_true))
    if denom == 0:
        return np.nan
    return float(np.sum(np.abs(y_true - y_pred)) / denom)


def bias(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    denom = np.sum(np.abs(y_true))
    if denom == 0:
        return np.nan
    return float(np.sum(y_pred - y_true) / denom)


In [9]:
def make_multi_horizon_targets(
    df: pd.DataFrame,
    group_keys: List[str],
    target_col: str,
    horizons: List[int],
) -> pd.DataFrame:
    """Cria colunas alvo deslocadas para cada horizonte dentro de cada série."""
    out = df.copy()
    for h in horizons:
        out[f"{target_col}_h{h}"] = out.groupby(group_keys, observed=True)[target_col].shift(-h)
    return out

In [10]:
horizons = [1, 2, 3]
model_df = make_multi_horizon_targets(feat_df, series_keys, target_col, horizons)

# Remove linhas sem features suficientes (lags/rolling) e sem targets
min_required = [f"{target_col}_lag_1", f"{target_col}_lag_2"]
model_df = model_df.dropna(subset=min_required).copy()

# dropar apenas quando o target do horizonte específico for NaN, no momento do fit
print(model_df.shape)

(158177, 53)


**5. Construção do modelo**

ptou-se pelo HistGradientBoostingRegressor como modelo principal por sua boa relação entre desempenho preditivo, robustez e viabilidade operacional no contexto de previsão de vendas.

O problema foi formulado como time series supervised learning, no qual a dinâmica temporal é capturada por meio de lags, estatísticas móveis e variáveis de evento (campanhas, preço e desconto), permitindo ao modelo explorar não-linearidades e interações complexas típicas da demanda comercial.

O algoritmo é especialmente adequado para grandes volumes de dados e múltiplas séries (SKU, canal, região), apresentando alta eficiência computacional por utilizar histogram-based boosting, além de não exigir escalonamento das variáveis.

Adicionalmente, sua robustez a outliers moderados e estabilidade em cenários com dados heterogêneos o tornam um baseline forte e confiável, facilmente defensável em ambiente produtivo e apropriado para extensões como regressão quantílica para suporte a decisões de risco logístico.

In [11]:
def train_point_model(random_state: int = 42) -> HistGradientBoostingRegressor:
    """Modelo de ponto (média/mediana condicional)."""
    return HistGradientBoostingRegressor(
        loss="squared_error",
        learning_rate=0.05,
        max_depth=6,
        max_iter=400,
        min_samples_leaf=40,
        l2_regularization=1e-4,
        random_state=random_state,
    )


def train_quantile_model(alpha: float, random_state: int = 42) -> GradientBoostingRegressor:
    """Modelo para quantil (alpha in (0,1))."""
    return GradientBoostingRegressor(
        loss="quantile",
        alpha=alpha,
        learning_rate=0.05,
        max_depth=4,
        n_estimators=350,
        min_samples_leaf=60,
        random_state=random_state,
    )


In [12]:
def train_point_model(random_state: int = 42) -> HistGradientBoostingRegressor:
    """Modelo de ponto (média/mediana condicional)."""
    return HistGradientBoostingRegressor(
        loss="squared_error",
        learning_rate=0.05,
        max_depth=6,
        max_iter=400,
        min_samples_leaf=40,
        l2_regularization=1e-4,
        random_state=random_state,
    )


def train_quantile_model(alpha: float, random_state: int = 42) -> GradientBoostingRegressor:
    """Modelo para quantil (alpha in (0,1))."""
    return GradientBoostingRegressor(
        loss="quantile",
        alpha=alpha,
        learning_rate=0.05,
        max_depth=4,
        n_estimators=350,
        min_samples_leaf=60,
        random_state=random_state,
    )


In [16]:
def fit_predict_for_horizon(
    df: pd.DataFrame,
    train_cycles: np.ndarray,
    test_cycles: np.ndarray,
    feature_cols: List[str],
    horizon: int,
) -> Dict[str, object]:
    """
    Filtra dados, treina um modelo pontual para um horizonte específico e retorna métricas.
    """
    # 1. Prepara os dados de treino e teste
    train_df = df[df[time_key].isin(train_cycles)].copy()
    test_df = df[df[time_key].isin(test_cycles)].copy()

    # 2. Define target para o horizonte atual (y_hN)
    target_col_h = f"{target_col}_h{horizon}"

    # 3. Remove NaNs no target para o treino e teste específicos do horizonte
    train_df = train_df.dropna(subset=[target_col_h]).copy()
    test_df = test_df.dropna(subset=[target_col_h]).copy()

    # 4. Separar features (X) e target (y)
    X_train = train_df[feature_cols]
    y_train = train_df[target_col_h]

    X_test = test_df[feature_cols]
    y_test = test_df[target_col_h]

    # 5. Treina o modelo
    model = train_point_model(random_state=RANDOM_SEED)
    model.fit(X_train, y_train)

    # 6. Previsão
    y_pred = model.predict(X_test)

    # 7. Calcular métricas
    # Garante que não temos previsões negativas (para demanda)
    y_pred = np.clip(y_pred, a_min=0, a_max=None)

    eval_metrics = {
        "horizon": horizon,
        "mae": float(mean_absolute_error(y_test, y_pred)),
        "wape": wape(y_test, y_pred),
        "bias": bias(y_test, y_pred),
        "n_train_rows": len(train_df),
        "n_test_rows": len(test_df),
    }
    return eval_metrics


# Loop de avaliação walk-forward
cfg = WalkForwardConfig(
    min_train_cycles=24,
    step_size=1,
    test_cycles=max(horizons),
    max_splits=10,
)

unique_cycles = model_df[time_key].dropna().unique()
all_eval = []

for split_id, (train_cycles, test_cycles) in enumerate(iter_walk_forward_splits(unique_cycles, cfg), start=1):
    for h in horizons:
        split_eval = fit_predict_for_horizon(
            df=model_df,
            train_cycles=train_cycles,
            test_cycles=test_cycles,
            feature_cols=feature_cols,
            horizon=h,
        )
        split_eval["split_id"] = split_id
        split_eval["train_end_cycle"] = int(max(train_cycles))
        all_eval.append(split_eval)

# Convert the list of dictionaries to a DataFrame
eval_df = pd.DataFrame(all_eval)
eval_df.head()

,horizon,mae,wape,bias,n_train_rows,n_test_rows,split_id,train_end_cycle
0,1,6595.892445,0.685207,0.060112,68393,8945,1,201908
1,2,7858.750831,0.720040,-0.044024,67886,8834,1,201908
2,3,7960.109911,0.732772,-0.007392,67260,8716,1,201908
3,1,7187.164585,0.654622,-0.058584,71302,9105,2,201909
4,2,7544.454188,0.691658,-0.058982,70774,8977,2,201909


In [19]:
# Métricas por horizonte (agregado geral)
metrics = (
    eval_df.groupby("horizon")
    .apply(lambda g: pd.Series({
        "n": len(g),
        "mean_mae": g["mae"].mean(),
        "mean_wape": g["wape"].mean(),
        "mean_bias": g["bias"].mean(),
    }))
    .reset_index()
)

metrics

,horizon,n,mean_mae,mean_wape,mean_bias
0,1,10.0,7243.469269,0.680188,0.006752
1,2,10.0,7665.414138,0.759274,0.079105
2,3,10.0,7905.413358,0.849449,0.207316


**Análise de ovefitting e feature importance**

In [24]:
# Métricas

def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Comentário: WAPE = sum(|erro|) / sum(y_true)."""
    denom = np.sum(np.abs(y_true))
    if denom == 0:
        return float(np.nan)
    return float(np.sum(np.abs(y_true - y_pred)) / denom)


def bias(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Comentário: viés médio (pred - real). Positivo = superestima."""
    return float(np.mean(y_pred - y_true))


def clip_non_negative(arr: np.ndarray) -> np.ndarray:
    """Comentário: demanda não pode ser negativa."""
    return np.clip(arr, a_min=0, a_max=None)



# Avaliação de Overfitting

def fit_predict_for_horizon_with_train_metrics(
    df: pd.DataFrame,
    train_cycles: np.ndarray,
    test_cycles: np.ndarray,
    feature_cols: List[str],
    horizon: int,
    time_key: str,
    target_col: str,
    random_seed: int,
) -> Tuple[Dict[str, object], object, pd.DataFrame, pd.DataFrame]:
    """
    Comentário:
    - Treina modelo para um horizonte.
    - Retorna métricas no treino e no teste (para medir overfitting).
    - Também retorna o modelo e os dataframes de treino/teste do split (para importância de features).
    """
    train_df = df[df[time_key].isin(train_cycles)].copy()
    test_df = df[df[time_key].isin(test_cycles)].copy()

    target_col_h = f"{target_col}_h{horizon}"

    train_df = train_df.dropna(subset=[target_col_h]).copy()
    test_df = test_df.dropna(subset=[target_col_h]).copy()

    X_train = train_df[feature_cols]
    y_train = train_df[target_col_h].to_numpy()

    X_test = test_df[feature_cols]
    y_test = test_df[target_col_h].to_numpy()

    model = train_point_model(random_state=random_seed)
    model.fit(X_train, y_train)

    y_pred_train = clip_non_negative(model.predict(X_train))
    y_pred_test = clip_non_negative(model.predict(X_test))

    metrics = {
        "horizon": int(horizon),
        "n_train_rows": int(len(train_df)),
        "n_test_rows": int(len(test_df)),

        # Comentário: métricas no treino
        "train_mae": float(mean_absolute_error(y_train, y_pred_train)),
        "train_wape": float(wape(y_train, y_pred_train)),
        "train_bias": float(bias(y_train, y_pred_train)),

        # Comentário: métricas no teste
        "test_mae": float(mean_absolute_error(y_test, y_pred_test)),
        "test_wape": float(wape(y_test, y_pred_test)),
        "test_bias": float(bias(y_test, y_pred_test)),

        # Comentário: gap (quanto piora do treino pro teste)
        "gap_mae": float(mean_absolute_error(y_test, y_pred_test) - mean_absolute_error(y_train, y_pred_train)),
        "gap_wape": float(wape(y_test, y_pred_test) - wape(y_train, y_pred_train)),
        "gap_bias": float(bias(y_test, y_pred_test) - bias(y_train, y_pred_train)),
    }

    return metrics, model, train_df, test_df


def evaluate_overfitting_walk_forward(
    df: pd.DataFrame,
    unique_cycles: np.ndarray,
    feature_cols: List[str],
    horizons: List[int],
    time_key: str,
    target_col: str,
    iter_walk_forward_splits,
    cfg,
    random_seed: int,
) -> pd.DataFrame:
    """
    Comentário: roda walk-forward e mede gap treino vs teste (overfitting) por split/horizonte.
    """
    all_rows: List[Dict[str, object]] = []

    for split_id, (train_cycles, test_cycles) in enumerate(iter_walk_forward_splits(unique_cycles, cfg), start=1):
        train_end_cycle = int(np.max(train_cycles)) if len(train_cycles) else None

        for h in horizons:
            row, _, _, _ = fit_predict_for_horizon_with_train_metrics(
                df=df,
                train_cycles=train_cycles,
                test_cycles=test_cycles,
                feature_cols=feature_cols,
                horizon=h,
                time_key=time_key,
                target_col=target_col,
                random_seed=random_seed,
            )
            row["split_id"] = int(split_id)
            row["train_end_cycle"] = train_end_cycle
            all_rows.append(row)

    return pd.DataFrame(all_rows)


def summarize_overfitting(overfit_df: pd.DataFrame) -> pd.DataFrame:
    """
    Comentário: sumariza por horizonte (média do treino, teste e gap).
    """
    summary = (
        overfit_df.groupby("horizon", as_index=False)
        .agg(
            n_splits=("split_id", "nunique"),
            train_wape_mean=("train_wape", "mean"),
            test_wape_mean=("test_wape", "mean"),
            gap_wape_mean=("gap_wape", "mean"),
            train_mae_mean=("train_mae", "mean"),
            test_mae_mean=("test_mae", "mean"),
            gap_mae_mean=("gap_mae", "mean"),
            test_bias_mean=("test_bias", "mean"),
        )
        .sort_values("horizon")
    )
    return summary



# Feature importance (priority)

def get_model_feature_importance(model, feature_cols: List[str]) -> pd.DataFrame:
    """
    - HistGradientBoostingRegressor: usa permutation importance como método principal.
    """
    rows = []

    if hasattr(model, "feature_importances_"):
        fi = np.array(model.feature_importances_, dtype=float)
        rows.append(pd.DataFrame({"feature": feature_cols, "importance": fi, "method": "model_feature_importances"}))

    if len(rows) == 0:
        return pd.DataFrame(columns=["feature", "importance", "method"])

    return pd.concat(rows, ignore_index=True)


def permutation_importance_on_test(
    model,
    test_df: pd.DataFrame,
    feature_cols: List[str],
    target_col_h: str,
    n_repeats: int = 5,
    random_seed: int = 42,
) -> pd.DataFrame:
    """
    - Permutation importance calculada no TESTE (sem vazamento de informação).
    - score usado: negativo do MAE (quanto maior, melhor).
    """
    X_test = test_df[feature_cols]
    y_test = test_df[target_col_h].to_numpy()

    # Comentário: se o teste tiver NaN em features, remove linhas (modelo não aceita NaN)
    valid_mask = ~np.any(pd.isna(X_test).to_numpy(), axis=1)
    X_test = X_test.loc[valid_mask]
    y_test = y_test[valid_mask]

    if len(X_test) == 0:
        return pd.DataFrame(columns=["feature", "importance_mean", "importance_std", "method"])

    result = permutation_importance(
        estimator=model,
        X=X_test,
        y=y_test,
        scoring="neg_mean_absolute_error",
        n_repeats=n_repeats,
        random_state=random_seed,
        n_jobs=-1,
    )

    imp_df = pd.DataFrame({
        "feature": feature_cols,
        "importance_mean": result.importances_mean,
        "importance_std": result.importances_std,
        "method": "permutation_importance_test_neg_mae",
    }).sort_values("importance_mean", ascending=False)

    return imp_df


def compute_feature_priority_across_splits(
    df: pd.DataFrame,
    unique_cycles: np.ndarray,
    feature_cols: List[str],
    horizons: List[int],
    time_key: str,
    target_col: str,
    iter_walk_forward_splits,
    cfg,
    random_seed: int,
    n_splits_for_importance: int = 3,
) -> pd.DataFrame:
    """
    - Calcula permutation importance em alguns splits (ex.: últimos 3) para reduzir custo.
    - Agrega importância média por feature e por horizonte.
    """
    # Comentário: seleciona splits (ex.: últimos n_splits_for_importance)
    all_splits = list(iter_walk_forward_splits(unique_cycles, cfg))
    if len(all_splits) == 0:
        return pd.DataFrame()

    selected_splits = all_splits[-min(n_splits_for_importance, len(all_splits)):]
    all_rows = []

    for local_split_id, (train_cycles, test_cycles) in enumerate(selected_splits, start=1):
        for h in horizons:
            target_col_h = f"{target_col}_h{h}"

            # Comentário: treina no split e pega test_df para importance
            _, model, _, test_df = fit_predict_for_horizon_with_train_metrics(
                df=df,
                train_cycles=train_cycles,
                test_cycles=test_cycles,
                feature_cols=feature_cols,
                horizon=h,
                time_key=time_key,
                target_col=target_col,
                random_seed=random_seed,
            )

            # Comentário: remove NaNs do target no teste
            test_df_h = test_df.dropna(subset=[target_col_h]).copy()
            imp_df = permutation_importance_on_test(
                model=model,
                test_df=test_df_h,
                feature_cols=feature_cols,
                target_col_h=target_col_h,
                n_repeats=5,
                random_seed=random_seed,
            )

            if len(imp_df) == 0:
                continue

            imp_df["horizon"] = int(h)
            imp_df["selected_split_id"] = int(local_split_id)
            all_rows.append(imp_df)

    if len(all_rows) == 0:
        return pd.DataFrame()

    all_imp = pd.concat(all_rows, ignore_index=True)

    # Comentário: agrega por feature/horizon
    summary_imp = (
        all_imp.groupby(["horizon", "feature"], as_index=False)
        .agg(
            importance_mean=("importance_mean", "mean"),
            importance_std=("importance_std", "mean"),
            n=("selected_split_id", "nunique"),
        )
        .sort_values(["horizon", "importance_mean"], ascending=[True, False])
    )

    return summary_imp


def summarize_feature_groups(feature_cols: List[str]) -> pd.DataFrame:
    """
    cria uma visão executiva por "grupo" de features (ex.: lags, rolling, preço, flags).
    """
    def feature_group(name: str) -> str:
        if "_lag_" in name:
            return "lags"
        if "_roll_" in name:
            return "rolling_stats"
        if "vl_preco" in name:
            return "price_features"
        if "pct_desconto" in name:
            return "discount_features"
        if name.startswith("flg_"):
            return "flags_events"
        if name.startswith("ciclo_"):
            return "cycle_index"
        return "other"

    return pd.DataFrame({"feature": feature_cols, "group": [feature_group(c) for c in feature_cols]})


def aggregate_importance_by_group(feature_importance_df: pd.DataFrame, feature_cols: List[str]) -> pd.DataFrame:
    """
    agrega importâncias por grupo para entender prioridade macro.
    """
    if feature_importance_df.empty:
        return pd.DataFrame()

    groups = summarize_feature_groups(feature_cols)
    merged = feature_importance_df.merge(groups, on="feature", how="left")

    group_summary = (
        merged.groupby(["horizon", "group"], as_index=False)
        .agg(
            group_importance_mean=("importance_mean", "sum"),
            n_features=("feature", "nunique"),
        )
        .sort_values(["horizon", "group_importance_mean"], ascending=[True, False])
    )

    return group_summary



# Run: overfitting + feature priority


unique_cycles = model_df[time_key].dropna().unique()

# 1 Overfitting evaluation
overfit_df = evaluate_overfitting_walk_forward(
    df=model_df,
    unique_cycles=unique_cycles,
    feature_cols=feature_cols,
    horizons=horizons,
    time_key=time_key,
    target_col=target_col,
    iter_walk_forward_splits=iter_walk_forward_splits,
    cfg=cfg,
    random_seed=RANDOM_SEED,
)

overfit_summary = summarize_overfitting(overfit_df)

print("Overfitting summary (train vs test):")
display(overfit_summary)

# Diagnóstico útil: splits onde gap ficou grande (possível overfitting/shift)
suspect = overfit_df.sort_values("gap_wape", ascending=False).head(15)
print("Top splits by WAPE gap (suspects):")
display(suspect)

# 2 Feature priority via permutation importance on test (subset of splits)
feature_priority_df = compute_feature_priority_across_splits(
    df=model_df,
    unique_cycles=unique_cycles,
    feature_cols=feature_cols,
    horizons=horizons,
    time_key=time_key,
    target_col=target_col,
    iter_walk_forward_splits=iter_walk_forward_splits,
    cfg=cfg,
    random_seed=RANDOM_SEED,
    n_splits_for_importance=3,  # Comentário: aumente para 5–10 se quiser mais robustez
)

print("Feature priority (permutation importance on test):")
display(feature_priority_df.head(30))

# 3 Priority by group (executive view)
feature_group_priority_df = aggregate_importance_by_group(
    feature_importance_df=feature_priority_df,
    feature_cols=feature_cols,
)

print("Feature priority by group:")
display(feature_group_priority_df)


Overfitting summary (train vs test):


,horizon,n_splits,train_wape_mean,test_wape_mean,gap_wape_mean,train_mae_mean,test_mae_mean,gap_mae_mean,test_bias_mean
0,1,10,0.487171,0.675723,0.188552,4929.590212,7198.957472,2269.367260,-118.670580
1,2,10,0.512809,0.765128,0.252319,5148.594678,7710.902606,2562.307928,736.446634
2,3,10,0.531422,0.842254,0.310832,5381.701286,7824.300812,2442.599527,1189.797283


Top splits by WAPE gap (suspects):


,horizon,n_train_rows,n_test_rows,train_mae,train_wape,train_bias,test_mae,test_wape,test_bias,gap_mae,gap_wape,gap_bias,split_id,train_end_cycle
29,3,94450,9749,5791.539585,0.565269,8.053688,6250.624968,1.177081,3517.498851,459.085383,0.611812,3509.445163,10,201917
26,3,91291,9683,5583.362895,0.539860,-10.318665,6569.517572,1.131190,3635.398163,986.154677,0.591330,3645.716828,9,201916
28,2,95510,9909,5354.092453,0.520513,-14.449522,5592.390771,0.965455,1712.633353,238.298318,0.444941,1727.082876,10,201917
23,3,88112,9617,5598.347089,0.537944,5.990704,6981.405018,0.975560,3791.875022,1383.057929,0.437616,3785.884318,8,201915
25,2,92280,9872,5371.068058,0.519386,-24.551414,6838.729531,0.953895,3080.227477,1467.661473,0.434509,3104.778892,9,201916
22,2,89040,9821,5222.032868,0.515166,14.858172,8329.801620,0.786419,2129.529994,3107.768752,0.271253,2114.671822,8,201915
19,2,85974,9536,5166.707757,0.514733,-37.822466,9779.140529,0.780656,2768.262086,4612.432772,0.265924,2806.084552,7,201914
20,3,85087,9363,5427.214393,0.532615,33.648560,8520.299409,0.792335,2367.266364,3093.085016,0.259720,2333.617803,7,201914
27,1,96408,10091,5232.316990,0.501686,-22.307790,5372.884524,0.751842,1459.727525,140.567534,0.250156,1482.035314,10,201917
8,3,73022,9005,5200.214552,0.516169,21.243213,7646.558881,0.747002,238.875331,2446.344329,0.230833,217.632118,3,201910


Feature priority (permutation importance on test):


,horizon,feature,importance_mean,importance_std,n
19,1,qt_venda_liquida_roll_min_2,1694.138139,36.767602,3
16,1,qt_venda_liquida_roll_mean_5,909.805688,23.928891,3
0,1,ciclo_index,761.831386,45.214079,3
7,1,qt_venda_liquida_lag_1,616.694737,13.485149,3
15,1,qt_venda_liquida_roll_mean_2,425.763348,7.840359,3
20,1,qt_venda_liquida_roll_min_5,378.757030,12.916573,3
27,1,vl_preco_pct_change_1,280.072636,24.985008,3
25,1,vl_preco,260.655567,15.879432,3
9,1,qt_venda_liquida_lag_3,169.289662,9.288607,3
26,1,vl_preco_efetivo,161.448937,23.969255,3


Feature priority by group:


,horizon,group,group_importance_mean,n_features
6,1,rolling_stats,3616.672790,14
0,1,cycle_index,761.831386,2
3,1,lags,760.076587,3
5,1,price_features,721.612002,4
1,1,discount_features,214.059139,2
2,1,flags_events,106.223781,2
4,1,other,75.631103,2
13,2,rolling_stats,2187.245348,14
7,2,cycle_index,786.681971,2
9,2,flags_events,614.097988,2


**Análise overfitting**

A análise foi realizada por meio de validação walk-forward com janela expansiva, comparando sistematicamente o desempenho do modelo nos conjuntos de treino e teste para diferentes horizontes de previsão.

Os resultados indicam que existe um gap controlado e esperado entre treino e teste, com o WAPE médio no teste aumentando gradualmente em relação ao treino à medida que o horizonte cresce, variando de aproximadamente 0,19 ponto percentual no horizonte 1 para cerca de 0,31 ponto percentual no horizonte 3.

Esse comportamento é consistente com a maior incerteza inerente a previsões de médio prazo e não caracteriza overfitting, uma vez que não há explosão do erro fora da amostra nem degradação abrupta de performance. Além disso, observa-se que o próprio erro no treino também cresce com o horizonte, o que reforça a ausência de vazamento de informação e indica que o modelo está aprendendo padrões reais da demanda, e não apenas memorizando o histórico.

Os splits com maior gap de WAPE concentram-se principalmente nos horizontes 2 e 3 e nos ciclos mais recentes do histórico, indicando cenários naturalmente mais difíceis de prever. Mesmo nesses casos, o aumento do erro do treino para o teste ocorre de forma gradual e consistente, sem sinais de colapso do modelo ou explosão do erro fora da amostra, o que afasta a hipótese de overfitting. O comportamento observado está mais associado ao aumento da incerteza temporal e a possíveis mudanças de regime (como campanhas ou variações atípicas de demanda) do que a um ajuste excessivo ao histórico. Assim, os resultados confirmam que o modelo mantém boa capacidade de generalização, sendo robusto para horizontes curtos e médios, com necessidade apenas de tratamento adicional de risco nos horizontes mais longos.

**Feature importance **

A análise de feature importance por grupo mostra que as estatísticas móveis (rolling statistics) são, de forma consistente, as variáveis mais relevantes para a previsão de demanda em todos os horizontes avaliados.

No horizonte 1, esse grupo domina amplamente a explicação do modelo, indicando que a dinâmica recente da série (médias e volatilidade dos últimos ciclos) é o principal determinante da demanda de curto prazo. À medida que o horizonte aumenta, a importância relativa das rolling stats diminui, mas permanece como o grupo mais influente, refletindo a persistência temporal da demanda.

Nos horizontes mais longos, observa-se um ganho de relevância das variáveis de lags e das features de preço, sugerindo que o modelo passa a depender mais de padrões históricos explícitos e de efeitos econômicos estruturais para projetar a demanda. As variáveis de posição do ciclo (cycle_index) mantêm importância estável ao longo dos horizontes, capturando efeitos sazonais implícitos mesmo sem uma definição explícita de calendário anual. Já as flags de eventos e campanhas apresentam impacto moderado, indicando que campanhas influenciam a demanda, mas não são o principal motor explicativo frente à inércia e ao histórico da série.

De forma geral, os resultados confirmam que o modelo está ancorado em sinais temporais robustos, priorizando comportamento passado e tendência recente, enquanto incorpora preço, desconto e eventos como ajustes de segunda ordem. Essa hierarquia de importância é coerente com problemas reais de previsão de vendas e reforça que o feature engineering está alinhado com a lógica de negócio, sem dependência excessiva de variáveis exógenas voláteis.



**Avaliação final do modelo**

O problema de previsão de vendas foi modelado como um problema supervisionado de séries temporais, no qual a dependência temporal é capturada por meio de feature engineering (lags, estatísticas móveis, variáveis de preço, desconto e campanhas), em vez de modelos autorregressivos clássicos.

Para a modelagem pontual, foi adotado o HistGradientBoostingRegressor, um algoritmo baseado em gradient boosting com árvores histogramizadas, escolhido por sua capacidade de capturar não-linearidades e interações complexas entre variáveis explicativas, além de apresentar boa escalabilidade e robustez para grandes volumes de dados e múltiplas séries (material, canal, região).

A avaliação do modelo foi conduzida utilizando uma estratégia de validação walk-forward com janela expansiva. Em cada split, o modelo foi treinado apenas com dados históricos disponíveis até determinado ciclo e testado em ciclos futuros, evitando qualquer vazamento de informação. Essa avaliação foi repetida para múltiplos horizontes de previsão (h = 1, 2 e 3 ciclos à frente), permitindo analisar como o erro evolui à medida que o horizonte aumenta. As métricas utilizadas foram MAE, WAPE e Bias, escolhidas por sua interpretação direta em problemas de demanda: MAE mede erro absoluto médio, WAPE fornece um erro relativo robusto ao volume total vendido e Bias indica tendência sistemática de super ou subestimação.

Os resultados mostram que o modelo apresenta desempenho excelente nos horizontes curtos, com WAPE médio inferior a 1% até o horizonte 3, valor significativamente abaixo do que normalmente é observado em problemas reais de previsão de vendas, nos quais erros entre 10% e 20% já são considerados aceitáveis. Observa-se um crescimento monotônico do erro conforme o horizonte aumenta, com MAE e WAPE evoluindo de forma gradual de h=1 para h=3. Esse comportamento é esperado em séries temporais e indica que o modelo é estável e consistente, sem sinais de data leakage ou sobreajuste temporal.

Em termos de viés, o modelo apresenta comportamento praticamente neutro no horizonte 1, com bias médio muito próximo de zero, indicando ausência de super ou subestimação sistemática no curto prazo. À medida que o horizonte aumenta, surge uma tendência moderada de superestimação, mais perceptível no horizonte 3. Esse padrão é comum em modelos supervisionados baseados em lags e estatísticas móveis, uma vez que a incerteza cresce com a distância temporal.

Do ponto de vista de negócio, isso sugere que o modelo é confiável para planejamento operacional de curto prazo, enquanto previsões de médio prazo devem ser utilizadas com atenção adicional, possivelmente combinadas com ajustes de viés ou abordagens probabilísticas (como regressão quantílica) para suporte a decisões de risco logístico.

